<a href="https://colab.research.google.com/github/wannleach/MAT-421/blob/main/ModuleC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Chapter 19: Root Finding**

*19.1: Root Finding Problem Statement*

In mathematics, finding the root (i.e. the value of x at which y=0 for a function) can be done either analytically or by approximation. Certain functions, however, prove difficult to solve by hand to find an exact solution and thus must be estimated. As shown below, the x-value for the function $f(x) = sin(x) - x$ is approximated for its root closest to $x=-8$ then verified to indeed solve for zero when plugged back into the function.

In [ ]:
import numpy as np
from scipy import optimize

f = lambda x: np.sin(x) - x # function to evaluate
r = optimize.fsolve(f, -8) # approximate root near x=-8
print("r =", r)

# Verify the solution is a root by plugging the calculated value into the function
result = f(r)
print("result=", result)

r = [-1.57829487e-08]
result= [0.]


However, this method will fail for functions with no real roots. For example, the function $f(x) = x^2 + 2$ has no x-values such that $f(x)=0$. Below, the computer cycles through so many iterations to reach some approximation, but this approximation is not verified to be a root, as the result of plugging our "root" back into the original function yields $result = 2.00000095 \neq 0$. The computer prints out an error regarding the failure of the iterations to solve for this value, acknowleding this result is not accurate. 

In [ ]:
import numpy as np
from scipy import optimize

f = lambda x: x**2 + 2 # function to evaluate
r = optimize.fsolve(f, -1) # approximate root near x=-1
print("r =", r)

# Verify the solution is a root by plugging the calculated value into the function
result = f(r)
print("result=", result)

r = [-0.00097656]
result= [2.00000095]


/usr/local/lib/python3.8/dist-packages/scipy/optimize/minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


*19.2: Tolerance*

As previously shown, functions do not always yield a desired root value, especially when considering the allotted precision a computer has to calculate values. However, as sometimes we still desire to find a value close to what this "desired root" could be, we can allow the computer a certain acceptable degree of error with which to find an accurate estimate. This degree of error is referred to as *tolerance*. In the following example, a quadratic function with no real root is first evaluated without tolerance, then with tolerance to yield an acceptable root.

In [ ]:
import numpy as np
from scipy import optimize

# original calculation w/o tolerance
f1 = lambda x: x**2 + 2 # function to evaluate
r1 = optimize.fsolve(f1, -1) # approximate root near x=-1

# calculation with tolerance
tol = 2 # tolerance; acceptable amount of error
f2 = lambda x: x**2 + 2 - tol # function
r2 = optimize.fsolve(f2, -1) # estimate root

# deciding if result 1 is within accepted tolerance level
if (np.absolute(f1(r1)) < tol):
  print("root with tolerance = ", r1)
else:
  print("root r1 = " + str(r1) + " is not within acceptable tolerance")

# deciding if result 2 is within accepted tolerance level
if (np.absolute(f2(r2)) < tol):
  print("root with tolerance = ", r2)
else:
  print("root r2 is not within acceptable tolerance")


root r1 = [-0.00097656] is not within acceptable tolerance
root with tolerance =  [-9.25035823e-09]


*19.3: Bisection Method*

The Bisection Method is a technique used to estimate the root value of a function between two designated values $a$ and $b$. This method is based on the Intermediate Value Theorm (IVT), which (intuitively) states that for a continuous function, if there are two outputs that map to opposite sign values (i.e. one is positive and one is negative) then there must exist a value between the two that equates to zero. The following function $my\_bisection$ uses IVT to estimate roots within a given tolerance level.First $a$ and $b$ are checked to have opposing signs to ensure a root exists between them. Next, the value of the midpoint between $a$ and $b$ is calculated for future evaluation using IVT. Then, for as long as the midpoint value falls outside of the accepted tolerance, the midpoint is continuously calculated to find the root, recursively using the previous midpoint as one of the bounds.


Below, the root between $x=0,2$ for the function $f(x) = sin(x) - cos(x)$ is approximated using the function $my\_bisection$ first with a tolerance level of $f(x) < 0.1$, then with $f(x) < 0.01$. With a lower tolerance for error, the second root found is more precise than the first. Because of this, when plugged back into the original function, the second estimated root is much closer to zero than the first.

In [ ]:
import numpy as np

# recursive function to approximate a root bounded by two points a and b within an alloted tolerance
def my_bisection(f, a, b, tol): 
    # approximates a root, R, of f bounded 
    # by a and b to within tolerance 
    # | f(m) | < tol with m the midpoint 
    # between a and b Recursive implementation
    
    # check if a and b bound a root
    if np.sign(f(a)) == np.sign(f(b)):
        raise Exception(
         "The scalars a and b do not bound a root")
        
    # get the midpoint
    m = (a + b)/2
    
    # find the approximated root w/in the tolerance
    if np.abs(f(m)) < tol:
        # stopping condition, report m as root
        return m
    elif np.sign(f(a)) == np.sign(f(m)):
        # case where m is an improvement on a. 
        # Make recursive call with a = m
        return my_bisection(f, m, b, tol)
    elif np.sign(f(b)) == np.sign(f(m)):
        # case where m is an improvement on b. 
        # Make recursive call with b = m
        return my_bisection(f, a, m, tol)

f = lambda x: np.sin(x) - np.cos(x)

# find root between x=0,2 with a tolerance level of f(x) < 0.1
r1 = my_bisection(f, 0, 2, 0.1)
print("r1 =", r1)

# find root between x=0,2 with a tolerance level of f(x) < 0.01
r01 = my_bisection(f, 0, 2, 0.01)
print("r01 =", r01)

print("f(r1) =", f(r1))
print("f(r01) =", f(r01))

r1 = 0.75
r01 = 0.78125
f(r1) = -0.050050108850486774
f(r01) = -0.005866372111545948


*19.4: Newton-Raphson Method*

The Newton-Raphson method utlizes tangent lines to make approximations of roots. Starting with a random value $x$ as our root "guess", we look for what function value the $x$-estimate yields and find the tangent line for that point on the function graph (so long as the function is continuous). The $x$-intercept for this line is then found, i.e. the root of this line to estimate the root of the overall function, which has been brought much closer to the actual value. Thus, the equation for the value of the derivative at a certain point on the function graph can be used to estimate the function's roots by treating it as a linear approximation.


Below, the original estimate of $x=1.4$ is compared to the accuracy of the new estimate using the newton-raphson method. As expected, the newton-raphson approximation is much closer to the actual root value than the original random guess.

In [50]:
import numpy as np

# newton-raphson method
f = lambda x: np.sin(x) - np.cos(x) # function to evaluate
f_prime = lambda x: np.cos(x) + np.sin(x) + f(1.4) # new function to estimate
newton_raphson = 1.4 - (f(1.4))/(f_prime(1.4)) # new estimate of root given an intitial estimate of x=1.4

print("original root estimate = 1.4")
print("newton_raphson root estimate =", newton_raphson) # new guess
print("f(1.4) =", f(1.4)) # original guess proximity
print("f(newton_raphson) = ", f(newton_raphson)) # check of proximity to real root

original root estimate = 1.4
newton_raphson root estimate = 0.9862383629159
f(1.4) = 0.8154825870882191
f(newton_raphson) =  0.282125298963106


*19.5: Root Finding in Python*

The Python library *scipy* has a built-in function *f-solve* to estimate the root of a function to a higher degree of accuracy than the previously mentioned techniques. This function takes in a number of parameters, including the function and the starting estimate, to use Jacobian approximation to compute the root(s) value(s).

In [51]:
from scipy.optimize import fsolve
import numpy as np

f = lambda x: np.cos(x) - np.sin(x) # function

fsolve(f, [2, 80]) # fsolve for the root(s) of f starting starting between estimates of 2 and 80

array([ 0.78539816, 79.3252145 ])